In [1]:
# Import glob, a module that helps with file management.
import glob
import pandas as pd
from spellchecker import SpellChecker
from pathlib import Path
from nltk import wordpunct_tokenize
from collections import Counter
import os
import re
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font
import numpy as np

In [2]:
# Define the output path and create the directory if it doesn't exist
outputpath = "./final"
outputfile_path = Path(outputpath)
outputfile_path.mkdir(exist_ok=True)

# Get the current working directory
texts_folder = Path.cwd()

# Find all .txt files in the current directory
texts_list = sorted(glob.glob("*.txt"))
print("Text files to be spellchecked:", texts_list)

current_directory = os.path.basename(os.getcwd())

print(f"current_directory = '{current_directory}'")

Text files to be spellchecked: ['Démonomanie I.1.txt', 'Démonomanie I.2.txt', 'Démonomanie I.3.txt', 'Démonomanie I.4.txt', 'Démonomanie I.5.txt', 'Démonomanie I.6.txt', 'Démonomanie I.7.txt', 'Démonomanie II.1.txt', 'Démonomanie II.2.txt', 'Démonomanie II.3.txt', 'Démonomanie II.4.txt', 'Démonomanie II.5.txt', 'Démonomanie II.6.txt', 'Démonomanie II.7.txt', 'Démonomanie II.8.txt', 'Démonomanie III.1.txt', 'Démonomanie III.2.txt', 'Démonomanie III.3.txt', 'Démonomanie III.4.txt', 'Démonomanie III.5.txt', 'Démonomanie III.6.txt', 'Démonomanie IV.1.txt', 'Démonomanie IV.2.txt', 'Démonomanie IV.3.txt', 'Démonomanie IV.4.txt', 'Démonomanie IV.5.txt', 'Démonomanie preface Repair.txt']
current_directory = 'Démonomanie'


In [3]:
# Load the spellchecker dictionary.
# Replace the language attribute with another 2 letter code
# to select another language. Options are: English - ‘en’, Spanish - ‘es’,
# French - ‘fr’, Portuguese - ‘pt’, German - ‘de’, Russian - ‘ru’.

spell = SpellChecker(language='fr')

In [4]:
# Initialize an empty list to collect data
data_list = []

with pd.ExcelWriter(f'{current_directory}_spellcheck_data.xlsx', engine='openpyxl') as writer:
    for txt_file in sorted(texts_folder.glob('*.txt')):
        
        # Extract the relevant parts of the file path
        parts = txt_file.parts[-3:]
        file_name = os.path.join(parts[0], parts[1], parts[2])
        
        # Open each text file and read text into `ocrText`
        with open(txt_file, 'r') as inputFile:
            ocrText = inputFile.read()
            
        # Join hyphenated words that are split between lines
        ocrText = ocrText.replace("-\n", "")
        
        # Tokenize the text
        tokens = wordpunct_tokenize(ocrText)
        
        # Lowercase all tokens and filter out non-alphabetic tokens
        tokens = [token.lower() for token in tokens if token.isalpha()]
        
        # Identify unknown words
        spell = SpellChecker()
        unknown = spell.unknown(tokens)
        
        # Calculate readability
        if len(unknown) != 0:
            readability = round(100 - (float(len(unknown)) / float(len(tokens)) * 100), 2)
        else:
            readability = 100
        
        # Sort the unknown words first by 'counts' descending and then by 'unknown_tokens' alphabetically
        sorted_unknown = sorted(Counter(unknown).items(), key=lambda item: (-item[1], item[0]))
        
        # Append the data for this file to the list
        data_list.append({
            "file_name": file_name,
            "token_count": len(tokens),
            "unknown_count": len(unknown),
            "readability": readability,
        })
        
        # Write the DataFrame to a new sheet in the Excel file
        pd.DataFrame([data_list[-1]]).to_excel(writer, sheet_name=os.path.splitext(txt_file.name)[0], index=False)
        
        # Write the unknown tokens and counts vertically
        sheet = writer.sheets[os.path.splitext(txt_file.name)[0]]
        start_row = 6  # Add two extra rows of space
        sheet.cell(row=start_row, column=1, value="unknown_tokens").font = Font(bold=True)
        sheet.cell(row=start_row, column=2, value="counts").font = Font(bold=True)
        for i, (token, count) in enumerate(sorted_unknown, start=start_row + 1):
            sheet.cell(row=i, column=1, value=token)
            sheet.cell(row=i, column=2, value=count)
        
        # Print a message indicating the file has been processed
        print(txt_file, "checked for readability.")
    
    # Create a DataFrame from the collected data
    df = pd.DataFrame(data_list)

  
    # Write the combined DataFrame to a summary sheet
    df.to_excel(writer, sheet_name='Summary', index=False)

# Load the workbook to adjust cell alignment
wb = load_workbook(f'{current_directory}_spellcheck_data.xlsx')

# Align cells in the summary sheet
summary_sheet = wb['Summary']
for row in summary_sheet.iter_rows(min_row=2, max_row=summary_sheet.max_row, min_col=1, max_col=4):
    for cell in row:
        cell.alignment = Alignment(vertical='top')

# Align cells in each individual sheet
for sheet_name in wb.sheetnames:
    if sheet_name != 'Summary':
        sheet = wb[sheet_name]
        for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row, min_col=1, max_col=4):
            for cell in row:
                cell.alignment = Alignment(vertical='top')

# Save the workbook with the updated alignment
wb.save(f'{current_directory}_spellcheck_data.xlsx')

df.head(20)

/home/lucas-jerusalimiec/Documents/OCR Text/Text/Sectionized/Démonomanie/Démonomanie I.1.txt checked for readability.
/home/lucas-jerusalimiec/Documents/OCR Text/Text/Sectionized/Démonomanie/Démonomanie I.2.txt checked for readability.
/home/lucas-jerusalimiec/Documents/OCR Text/Text/Sectionized/Démonomanie/Démonomanie I.3.txt checked for readability.
/home/lucas-jerusalimiec/Documents/OCR Text/Text/Sectionized/Démonomanie/Démonomanie I.4.txt checked for readability.
/home/lucas-jerusalimiec/Documents/OCR Text/Text/Sectionized/Démonomanie/Démonomanie I.5.txt checked for readability.
/home/lucas-jerusalimiec/Documents/OCR Text/Text/Sectionized/Démonomanie/Démonomanie I.6.txt checked for readability.
/home/lucas-jerusalimiec/Documents/OCR Text/Text/Sectionized/Démonomanie/Démonomanie I.7.txt checked for readability.
/home/lucas-jerusalimiec/Documents/OCR Text/Text/Sectionized/Démonomanie/Démonomanie II.1.txt checked for readability.
/home/lucas-jerusalimiec/Documents/OCR Text/Text/Sectio

,file_name,token_count,unknown_count,readability
0,Sectionized/Démonomanie/Démonomanie I.1.txt,2931,1218,58.44
1,Sectionized/Démonomanie/Démonomanie I.2.txt,3639,1371,62.32
2,Sectionized/Démonomanie/Démonomanie I.3.txt,3189,1286,59.67
3,Sectionized/Démonomanie/Démonomanie I.4.txt,3574,1373,61.58
4,Sectionized/Démonomanie/Démonomanie I.5.txt,6484,2274,64.93
5,Sectionized/Démonomanie/Démonomanie I.6.txt,3195,1335,58.22
6,Sectionized/Démonomanie/Démonomanie I.7.txt,1467,659,55.08
7,Sectionized/Démonomanie/Démonomanie II.1.txt,3553,1420,60.03
8,Sectionized/Démonomanie/Démonomanie II.2.txt,4481,1733,61.33
9,Sectionized/Démonomanie/Démonomanie II.3.txt,4955,2053,58.57


In [5]:
unknown_words = [
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ainfi', 'ariftote', 'auffi', 'aufli', 'auons', 'auoient',
    'brufler', 'bruflé', 'bruflés', 'bruflées', 'bruflerent', 'brufloient',
    'caufe', 'caufes', 'cefte', 'celeftes', 'celte', 'chan- \ngement', 
    'chofe', 'chofes', 'ciuil',
    'confefla', 'confefle', 'confeflé', 'confefler', 'confeflerent', 'confeflion', 'conveflions', 'confefloyent',
    'deffus', 'deflus', 'deſquels', 'def- \nquels', 'dia ble', 'dia- \n\nbles', 
    'dia- \nble', 'dia- \nbolique', 'dia- \nboliques', 'diabo- \nliques', ' difant', 'difoit',
    'diuin', 'diuins', 'diuine', 'diuines', 'diuinité', 'doibt',
    'efcript', 'efcripte', 'efcriptes', 'efcripture', 'efprit', 'efprits', 'eftà', 'empe- \nreur', ' enla ', 'enlaloy', ' eten ', 'expe- \nrience',
    'faiét', 'faifant', 'faincte', 'fans', 'fix', 'fem- \nblable', 
    'fem- \nblables', 'feulement', 'fouuent', 'fupernaturelles',
    'hiftoire', 'hiftoires', 'honteufes', 'hu- \nmainc', 'hu- \nmaine',
    'imagi- \nner', 'l\'imagi- \nnation', 
    ' laloy ', 'lefe', 'leuiathan', 'liure',
    'maiftre', 'male- \nfices', 'maling', 'malings', 'mefme', 'mefmes', 'monftré', 'mouuement', 'mou- \nrir', 'moyés',
    'na- \nture', 'natu- \nre', 'natu- \nrc', 
    'natu- \nrelle', 'œuures', 'ordi- \nnaire', 'ordi- naires',
    'parlemens', 'paruenire', 'perfonne', 'perfonnes', 'perfonnage', 'perfonnages',
    'peuuent', 'plufieurs', 'plu- \n\nfieurs', 
    'plu- \nficurs', 'plu- \nfieurs', 'prefomption', 'prefomptions', 'preuue',
    'pro- \nphetie', 'pro- \npheties', 'pro- \nphetize', 
    'pro- \nphetes', 'puiflance', 'qvatriesme', 'raifon', 'sor- \ncelleries', 'sor- \ncicr', 'sor- \nciet', 'sor- \nciere', 
    'sor- \ncier', 'trouua', 'trouue', 'trouué', 'trouuées', 'vns'
]


known_words = [
    '', '', '', '', '', '', '', '', '', '', 'ainsi', 'aristote', 'aussi', 'aussi', 'avons', 'avoient',
    'brûler', 'brûlé', 'brûlés', 'brûlées', 'brûlerent', 'brûloient',
    'cause', 'causes', 'ceste', 'celestes', 'ceste', 'changement', 
    'chose', 'choses', 'civil',
    'confessa', 'confesse', 'confessé', 'confesser', 'confessèrent', 'confession', 'confessions', 'confessaient',
    'dessus', 'dessus', 'desquels', 'desquels', 'diable', 'diables', 
    'diable', 'diabolique', 'diaboliques', 'diaboliques', ' disant', 'disoit',
    'divin', 'divins', 'divine', 'divines', 'divinité', 'doit',
    'escript', 'escripte', 'escriptes', 'escripture', 'esprit', 'esprits', 'cest à', 'empereur', ' en la', 'en la loy', ' et en ', 'experience',
    'fait', 'faisant', 'saincte', 'sans', 'six', 'semblable',
    'semblables', 'seulement', 'souvent', 'supernaturelles',
    'histoire', 'histoires', 'honteuses', 'humaine', 'humaine',
    'imaginer', 'l\'imagination',
    'la loy ', 'lese', 'leviathan', 'livre',
    'maistre', 'malefices', 'malin', 'malins', 'mesme', 'mesmes', 'monstré', 'mouvement', 'mourir', 'moyens',
    'nature', 'nature', 'nature',
    'naturelle', 'oeuvres', 'ordinaire', 'ordinaires',
    'parlements', 'parvenir', 'personne', 'personnes', 'personnage', 'personnages',
    'peuvent', 'plusieurs', 'plusieurs',
    'plusieurs', 'plusieurs', 'presomption', 'presomptions', 'preuve',
    'prophetie', 'propheties', 'prophetize',
    'prophetes', 'puissance', 'quatriesme', 'raison', 'sorcelleries', 'sorcier', 'sorcier', 'sorciere',
    'sorcier', 'trouva', 'trouve', 'trouvé', 'trouvées', 'uns'
]

In [6]:
first_file_iteration = True  # Flag to control the print statement

for file in texts_list:
    
    # Identify the output file path for each text file
    outputfile = f'{outputfile_path}/{Path(file).stem}_corrected.txt'
    
    # Open a file in "read" (r) mode
    with open(file, "r") as text:
        # Read in the contents of that file
        word_correction = text.read()

    word_correction = word_correction.lower()
    
    # Find instances of an unknown word and replace with a known word
    for i in range(len(known_words)):
        unknown_word = unknown_words[i]
        known_word = known_words[i]
        word_correction = word_correction.replace(unknown_word, known_word)
        
        if first_file_iteration:
            print(f"All instances of '{unknown_word}' replaced with '{known_word}'.")
    
    # Open the file in "write" (w) mode
    with open(outputfile, "w") as file:
        # Add the changed word into the reopened file
        file.write(word_correction)
    
    print(f"Corrected file saved as {outputfile}.")
    first_file_iteration = False  # Disable the flag after the first file

All instances of '0' replaced with ''.
All instances of '1' replaced with ''.
All instances of '2' replaced with ''.
All instances of '3' replaced with ''.
All instances of '4' replaced with ''.
All instances of '5' replaced with ''.
All instances of '6' replaced with ''.
All instances of '7' replaced with ''.
All instances of '8' replaced with ''.
All instances of '9' replaced with ''.
All instances of 'ainfi' replaced with 'ainsi'.
All instances of 'ariftote' replaced with 'aristote'.
All instances of 'auffi' replaced with 'aussi'.
All instances of 'aufli' replaced with 'aussi'.
All instances of 'auons' replaced with 'avons'.
All instances of 'auoient' replaced with 'avoient'.
All instances of 'brufler' replaced with 'brûler'.
All instances of 'bruflé' replaced with 'brûlé'.
All instances of 'bruflés' replaced with 'brûlés'.
All instances of 'bruflées' replaced with 'brûlées'.
All instances of 'bruflerent' replaced with 'brûlerent'.
All instances of 'brufloient' replaced with 'brûloi

In [7]:
import locale

# Set the locale to your desired setting (e.g., 'fr_FR.UTF-8' for French)
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

# Assume spell is a previously defined spell checker instance
# outputfile_path is the directory path to save the Excel file
# outputpath is the directory path containing the .txt files

# Create an Excel writer object
with pd.ExcelWriter(f'{outputfile_path}/{Path.cwd().name}_corrected_spellcheck_data.xlsx', engine='openpyxl') as writer:

    # Get list of files and sort them alphabetically using locale-aware sorting
    files = [f for f in os.listdir(outputpath) if f.endswith('.txt')]
    sorted_files = sorted(files, key=locale.strxfrm)

    for filename in sorted_files:
        file_path = os.path.join(outputpath, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text_data = file.read()

        text_data = text_data.replace("-\n", "")

        words = wordpunct_tokenize(text_data)

        misspelled = spell.unknown(words)

        if len(misspelled) != 0:
            readability = round(100 - (float(len(misspelled)) / float(len(words)) * 100), 2)
        else:
            readability = 100

        # Count the frequency of each misspelled word
        word_counts = Counter(misspelled)

        # Sort word_counts first by 'count' descending and then by 'word' alphabetically
        sorted_word_counts = sorted(word_counts.items(), key=lambda item: (-item[1], item[0]))

        # Create a DataFrame and sort by frequency
        misspelled_df = pd.DataFrame(sorted_word_counts, columns=['word', 'count'])

        # Add readability as the first row
        readability_header = pd.DataFrame([["Readability", readability]], columns=['word', 'count'])
        blank_rows = pd.DataFrame([["", ""]], columns=['word', 'count'])
        header_row = pd.DataFrame([["word", "count"]], columns=['word', 'count'])
        misspelled_df = pd.concat([readability_header, blank_rows, header_row, misspelled_df], ignore_index=True)

        # Write the DataFrame to a new sheet in the Excel file
        sheet_name = os.path.splitext(filename)[0]
        misspelled_df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

        print(f'Spellchecked {filename}. Readability = {readability}')

print(f"All data combined into {outputfile_path}/{Path.cwd().name}_corrected_spellcheck_data.xlsx")


Spellchecked Démonomanie I.1_corrected.txt. Readability = 70.5
Spellchecked Démonomanie I.2_corrected.txt. Readability = 71.72
Spellchecked Démonomanie I.3_corrected.txt. Readability = 69.59
Spellchecked Démonomanie I.4_corrected.txt. Readability = 70.89
Spellchecked Démonomanie I.5_corrected.txt. Readability = 74.36
Spellchecked Démonomanie I.6_corrected.txt. Readability = 68.5
Spellchecked Démonomanie I.7_corrected.txt. Readability = 65.91
Spellchecked Démonomanie II.1_corrected.txt. Readability = 70.0
Spellchecked Démonomanie II.2_corrected.txt. Readability = 71.2
Spellchecked Démonomanie II.3_corrected.txt. Readability = 68.39
Spellchecked Démonomanie II.4_corrected.txt. Readability = 72.65
Spellchecked Démonomanie II.5_corrected.txt. Readability = 64.44
Spellchecked Démonomanie II.6_corrected.txt. Readability = 75.78
Spellchecked Démonomanie II.7_corrected.txt. Readability = 66.48
Spellchecked Démonomanie II.8_corrected.txt. Readability = 71.1
Spellchecked Démonomanie III.1_correc

/home/lucas-jerusalimiec/.local/lib/python3.12/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


All data combined into final/Démonomanie_corrected_spellcheck_data.xlsx
